# Working with AWS Dynamo DB using Python

## Dynamo DB Overview
Amazon DynamoDB is a NoSQL database that supports key-value and document data models. It delivers single-digit millisecond performance at any scale. It's a fully managed, multiregion, multimaster, durable database with built-in security, backup and restore, and in-memory caching for internet-scale applications

Click [here](https://aws.amazon.com/dynamodb/) for more information

If you have an Amazon Free Tier account, you can use this notebook to create a **jobs1** table in the DynamoDB. Use it to insert and query the data from the table.

## Python and Dynamo DB

We need to import **boto3** package to be able to work with Dynamo DB using Python

Click [here](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.html) for the boto3 Dynamo DB documentation

In [1]:
from __future__ import print_function
import boto3
from boto3.dynamodb.conditions import Key, Attr
import os
import pandas as pd
import json
import time
from decimal import Decimal

### Create a resource representing the Amazon DynamoDB

We need to specify the following:
1. resource name - "dynamodb"
2. region name
3. aws_access_key_id
4. aws_secret_access_key

I haven't added the access key or secret key in the script for security reasons. If you add them, the statement would look something like this
dynamodb = boto3.resource("dynamodb", region_name = "us-west-1", aws_access_key_id = "XXX", aws_secret_access_key = "XXX")

Alternatively, one can set up the environment to contain the keys and utilize those environmental variables when creating the resouce.

In [2]:
dynamodb = boto3.resource("dynamodb", region_name = "us-west-1")

### Create table with partition and sort keys

DynamoDB has two kinds of primary keys. Other than these keys, the table is schemaless.
1. Partition Key
2. Sort Key

These keys can be use to simluate the primary key and composite primary key concepts in relation database.
1. Partition key - used as input to an internal hash function. The output from the hash function determines the partition in which the item will be stored. In a table that has only a partition key, no two items can have the same partition key value.

2. Partition and Sort key - partition key value is used as input to an internal hash function. The output from the hash function determines the partition in which the item will be stored. All items with the same partition key value are stored together, in sorted order by sort key value. In a table that has a partition key and a sort key, it's possible for two items to have the same partition key value. However, those two items must have different sort key values.

We also need to specify the read and write capacity units when creating the table. I have used 20 units for each in the example as the maximum limit available for AWS free tier is 25.

* Read Capacity Units - One read capacity unit represents one strongly consistent read per second, or two eventually consistent reads per second, for an item up to 4 KB in size.

* Write Capacity Units - One write capacity unit represents one write per second for an item up to 1 KB in size. If you need to write an item that is larger than 1 KB, DynamoDB must consume additional write capacity units.

Click [here](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.ReadWriteCapacityMode.html) to learn more

In **jobs1** table, we are creating a composite primary key using the job_title and a combined attribute of company, location information. As these three information uniquely identifies each item in the table. Since we can use only two attributes to create a unique identifier for each item in DynamoDB, we are combining the company name and job location data into a single value separated by an underscore.

In [3]:
table = dynamodb.create_table(
    TableName='jobs1',
    KeySchema=[
        {
            'AttributeName': 'job_title',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'company_location',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'job_title',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'company_location',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 20,
        'WriteCapacityUnits': 20
    }
)


### Check the table schema

In [4]:
dynamoDBClient = boto3.client('dynamodb')
table = dynamoDBClient.describe_table(
    TableName='jobs1'
)
print(table)

{'Table': {'AttributeDefinitions': [{'AttributeName': 'company_location', 'AttributeType': 'S'}, {'AttributeName': 'job_title', 'AttributeType': 'S'}], 'TableName': 'jobs1', 'KeySchema': [{'AttributeName': 'job_title', 'KeyType': 'HASH'}, {'AttributeName': 'company_location', 'KeyType': 'RANGE'}], 'TableStatus': 'ACTIVE', 'CreationDateTime': datetime.datetime(2020, 4, 2, 23, 8, 8, 5000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 20, 'WriteCapacityUnits': 20}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:us-west-1:764210939372:table/jobs1', 'TableId': 'e32a08ab-b994-4c00-b2b4-3b0f91991e59'}, 'ResponseMetadata': {'RequestId': 'EA345A830UL7GDBDMBF2NGA6Q7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 03 Apr 2020 06:08:12 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '585', 'connection': 'keep-alive', 'x-amzn-requestid': 'EA345A830UL7GDBDMB

### Select an existing table from the DynamoDB

In [5]:
table = dynamodb.Table("jobs1")

### Reading data from file

In [6]:
# Reads the job listing data collected from indeed website
data1 = pd.read_csv("indeed_job_listings.csv", usecols = [1, 2, 3, 4, 5, 6, 7, 8, 9], engine= 'python')

# Creates the range key by combining company and location columns using a "_"
data1["company_location"] = data1["Company"] + "_" + data1["City"]

# Renaming the columns of the dataframe
data1.columns = ["job_title", "company", "reviews", "description", "salary", "city", "state", "zip_code", "source", "company_location"]

In [7]:
data1.head()

,job_title,company,reviews,description,salary,city,state,zip_code,source,company_location
0,Data Analyst,ABC,10 reviews,With one application you can be considered for...,NaN,Seattle,WA,NaN,Indeed,ABC_Seattle
1,Business Analyst,BCD,680 reviews,Cloud Hardware Infrastructure and Engineering ...,NaN,Redmond,WA,NaN,Indeed,BCD_Redmond
2,ML Data Associate,XYZ,759 reviews,Basic Qualifications\nBachelors degree or comm...,NaN,Seattle,WA,NaN,Indeed,XYZ_Seattle
3,Data Analyst,WXY,NaN,Looking for a Data Analyst to join our Strateg...,NaN,Boston,MA,2116.0,Indeed,WXY_Boston
4,Business Analyst,DEF,NaN,Looking to add experienced and entry-level imp...,"$65,553.00 to $120,000.00 /year",Marlborough,MA,1752.0,Indeed,DEF_Marlborough


In [8]:
# Converts the rows into json string
data1 = data1.to_json(orient='records')

# Convert json string to json object
job_data = json.loads(data1, parse_float = Decimal)

In [9]:
len(job_data)

11

### Uploading data to the database using batch writer

Batch write can be used to write data to the DynamoDB in batches instead of item by item

In [10]:
with table.batch_writer(overwrite_by_pkeys=['job_title', 'company_location']) as batch:
    for i in range(len(job_data)):
        print(i)
        batch.put_item(Item = job_data[i])
        # wait time is used between each records to reduce the write frequency in the database
        time.sleep(0.5)

0
1
2
3
4
5
6
7
8
9
10


### Add an item to the table

The attributes that we mentioned as partition and sort keys when creating the table must be included when adding an item to the table. All other attributes are optional.

In [11]:
table.put_item(
    Item = {
        "job_title" : "Data Analyst",
        "company_location" : "ano_Seattle",
        "company" : "ano",
        "location" : "Seattle",
        "city" : "WA",
        "description" : "With one application you can be considered for thousands of tech roles from leading companies on Seen. Seen by Indeed is a free service that connects you to opportunities that take you further in your career."
    })

{'ResponseMetadata': {'RequestId': 'NRFTHJL7EMMK17I1OM0S1QS8LBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 03 Apr 2020 06:08:31 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'NRFTHJL7EMMK17I1OM0S1QS8LBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
# Print out the item count in the table. It takes ~6 hrs for the correct count to reflect. 
#I just inserted the records so we are still seeing 0 in the count
print(table.item_count)

0


## Get an item from the table

We must specify the primary key values to read any item from jobs1 table

In [13]:
response = table.get_item(
    Key={
        "job_title": "Business Analyst",
        "company_location": "BCD_Redmond"
    }
)
print(response['Item'])

{'city': 'Redmond', 'job_title': 'Business Analyst', 'company': 'BCD', 'salary': None, 'zip_code': None, 'source': 'Indeed', 'reviews': '680 reviews', 'description': 'Cloud Hardware Infrastructure and Engineering Group is the engine that powers the cloud services. The team is responsible for designing, developing and maintaining the core infrastructure and foundational technologies for over 200 online businesses. We focus on smart growth with an emphasis on automation, data driven engineering, costeffectiveness and environmental sustainability. We are looking for a passionate, high energy individual to help build the End to End lifecycle that powers the worlds largest online services. This is a great opportunity to join a dynamic team and influence the way one of the worlds largest and fastest growing cloud environments is built and supported. As a critical part of the Cloud Hardware Infrastructure and Engineering team, this role will help define how we improve overall delivery and dri

### Query from the table using key

We can query the items from the table only using the partition key

In [14]:
response = table.query(
    KeyConditionExpression = Key("job_title").eq("Data Analyst"))

In [15]:
for i in response['Items']:
    print(i['job_title'], ":", i['company'])

Data Analyst : ABC
Data Analyst : WXY
Data Analyst : ano


### Screenshot of the table from AWS

![title](table_data.png)

#### This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.